# Practical Lesson 4: Product Allocation to Factories
<sup>Example from chapter 8.3 of the book `Introduction to Operations Research` by `Hillier and Lieberman`.</sup>

### Problem Description
The Better Products Company has decided to start producing four new products using three factories that currently have excess production capacity. The products require a comparable production effort per unit, so the available production capacity of the factories is measured by the number of units of any product that can be produced daily, as shown in Table 1. Table 2 provides the daily production rate required to meet projected sales. Each factory is capable of producing any of these products, except Factory 2, which cannot manufacture product 3. The variable costs per unit of each product differ from factory to factory, as shown in Table 3.

Table 1: Available production capacity, per product unit

| | Factory 1 | Factory 2 | Factory 3 |
|:---|:---:|:---:|:---:|
| Available capacity | 75 | 75 | 45 |

Table 2: Daily production demand

| | Product 1 | Product 2 | Product 3 | Product 4 |
|:---|:---:|:---:|:---:|:---:|
| Demand | 20 | 30 | 30 | 40 |

Table 3: Unit cost per product

| | Product 1 | Product 2 | Product 3 | Product 4 |
|:---|:---:|:---:|:---:|:---:|
| Factory 1 | 41 | 27 | 28 | 24 |
| Factory 2 | 40 | 29 | - | 23 |
| Factory 3 | 37 | 30 | 27 | 21 |

Management needs to make a decision on how to divide the manufacturing of products among the factories, and has decided to allow the production of the same product to be split across more than one factory.

Write a model to determine which factories will produce which products and at what total cost.

### Solution

In [11]:
from mip import *

# solves the model and shows the variable values
def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value:.2f}\n")

    print("Solution:")
    for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

# saves the model to an lp file and shows its contents
def save(model, filename):
    model.write(filename) # saves the model to a file
    with open(filename, "r") as f: # reads and displays the file contents
        print(f.read())

### Exercise 2

Redo the previous model, this time preventing the division of products among factories. That is, each product must be entirely produced in a single factory, to eliminate hidden costs associated with splitting production. Each factory must be assigned at least one product.

In [12]:
# We want to minimize the production cost while meeting the demand
model = Model(sense=MINIMIZE, solver_name=CBC)

capacities = [75, 75, 45]
demands = [20, 30, 30, 40]
costs = [[41, 27, 28, 24],
         [40, 29,  0, 23],
         [37, 30, 27, 21]]

num_factories = 3
num_products = 4

# x[i][j] := 1 if product j is produced in factory i
x = [[model.add_var(var_type=INTEGER,
                    name=f"x_{i}_{j}", lb=0.0) for j in range(num_products)]
     for i in range(num_factories)]

# Objective function
model.objective = xsum(costs[i][j] * x[i][j] * demands[j]
                        for i in range(num_factories) for j in range(num_products))

# Capacity constraint
for i in range(num_factories):
    model += xsum(x[i][j] * demands[j] for j in range(num_products)) <= capacities[i]

# Each product can only be produced by one factory
for j in range(num_products):
    model += xsum(x[i][j] for i in range(num_factories)) == 1

# Binary constraints
for i in range(num_factories):
    for j in range(num_products):
        model += x[i][j] <= 1

# Factory 2 cannot produce product 3
model += x[1][2] == 0

save(model, "../data/lesson4.lp")

\Problem name: 

Minimize
OBJROW: 820 x_0_0 + 810 x_0_1 + 840 x_0_2 + 960 x_0_3 + 800 x_1_0 + 870 x_1_1 + 920 x_1_3 + 740 x_2_0 + 900 x_2_1 + 810 x_2_2
 + 840 x_2_3
Subject To
constr(0):  20 x_0_0 + 30 x_0_1 + 30 x_0_2 + 40 x_0_3 <= 75
constr(1):  20 x_1_0 + 30 x_1_1 + 30 x_1_2 + 40 x_1_3 <= 75
constr(2):  20 x_2_0 + 30 x_2_1 + 30 x_2_2 + 40 x_2_3 <= 45
constr(3):  x_0_0 + x_1_0 + x_2_0 = 1
constr(4):  x_0_1 + x_1_1 + x_2_1 = 1
constr(5):  x_0_2 + x_1_2 + x_2_2 = 1
constr(6):  x_0_3 + x_1_3 + x_2_3 = 1
constr(7):  x_0_0 <= 1
constr(8):  x_0_1 <= 1
constr(9):  x_0_2 <= 1
constr(10):  x_0_3 <= 1
constr(11):  x_1_0 <= 1
constr(12):  x_1_1 <= 1
constr(13):  x_1_2 <= 1
constr(14):  x_1_3 <= 1
constr(15):  x_2_0 <= 1
constr(16):  x_2_1 <= 1
constr(17):  x_2_2 <= 1
constr(18):  x_2_3 <= 1
constr(19):  x_1_2 = 0
Bounds
Integers
x_0_0 x_0_1 x_0_2 x_0_3 x_1_0 x_1_1 x_1_2 x_1_3 x_2_0 x_2_1 
x_2_2 x_2_3 
End



In [14]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3290.00

Solution:
x_0_0 = 0.00
x_0_1 = 1.00
x_0_2 = 1.00
x_0_3 = 0.00
x_1_0 = 1.00
x_1_1 = 0.00
x_1_2 = 0.00
x_1_3 = 0.00
x_2_0 = 0.00
x_2_1 = 0.00
x_2_2 = 0.00
x_2_3 = 1.00
